In [ ]:
import pandas as pd

In [ ]:
raw_h = pd.read_csv ('/Users/linze/PycharmProjects/untitled/datacleaning/H.csv')
raw_immune = pd.read_csv ('/Users/linze/PycharmProjects/untitled/datacleaning/immune.csv')
raw_go = pd.read_csv ('/Users/linze/PycharmProjects/untitled/datacleaning/GO.csv')


In [89]:
def stripe_leading_edge(input):
    input=input.replace("c", "")
    input=input.replace("(", "")
    input=input.replace(")", "")
    input=input.replace("\"", "")
    edge_list=input.split(", ")
    out_list = []
    for each in edge_list:
        each = each
        out_list.append(each)
    return out_list


def get_gene_dict(dataset):
    gene_list = []
    for index_1, row_1 in dataset.iterrows ():
        gene_list+=stripe_leading_edge(row_1['leadingEdge'])
    mylist = list(set(gene_list))
    dict_gene={}
    for each in mylist:
        dict_gene[each]=gene_list.count(each)
    return dict_gene


def get_pathway_size_list(dataset):
    size_list = []
    for index_1, row_1 in dataset.iterrows ():
        size_list.append(len(stripe_leading_edge(row_1['leadingEdge'])))
    return size_list 


def get_pathway_dict(dataset):
    dict_path={}
    gene_list = []
    for index_1, row_1 in dataset.iterrows ():
        gene_list+=stripe_leading_edge(row_1['leadingEdge'])
    gene_list = list(set(gene_list))
    for index_1, row_1 in dataset.iterrows ():
        dict_path[row_1['pathway']]=stripe_leading_edge(row_1['leadingEdge'])
    return gene_list, dict_path

In [90]:
dict_gene_h=get_gene_dict(raw_h)
dict_gene_go=get_gene_dict(raw_go)
dict_gene_immune=get_gene_dict(raw_immune)

In [100]:
pathway_h=raw_h['pathway']
enrich_h = [x * -1 for x in raw_h['NES']]
size_h=get_pathway_size_list(raw_h)
pathway_immune=raw_immune['pathway']
enrich_immune = [x * -1 for x in raw_immune['NES']]
size_immune=get_pathway_size_list(raw_immune)
pathway_go=raw_go['pathway']
enrich_go = [x * -1 for x in raw_go['NES']]
size_go=get_pathway_size_list(raw_go)

In [101]:
heat_list_h, heat_dict_h=get_pathway_dict(raw_h)
heat_list_immune, heat_dict_immune=get_pathway_dict(raw_immune)
heat_list_go, heat_dict_go=get_pathway_dict(raw_go)

In [102]:
def get_one_hot(gene_list, pathway_dict, pathway_list):
    one_hot_dict={}
    for each_gene in gene_list:
        one_hot_list=[]
        for each_path in pathway_list:
            if each_gene in pathway_dict[each_path]:
                one_hot_list.append(1)
            else:
                one_hot_list.append(0)
        one_hot_dict[each_gene]=one_hot_list
    return one_hot_dict

Create dataframe

In [103]:
H_gene_occ=pd.DataFrame(dict_gene_h,index=[0])
immune_gene_occ=pd.DataFrame(dict_gene_immune,index=[0])
go_gene_occ=pd.DataFrame(dict_gene_go,index=[0])

In [104]:
H_pathway_stat=pd.DataFrame({'pathway': pathway_h, 'NES': enrich_h, 'size':size_h})
immune_pathway_stat=pd.DataFrame({'pathway': pathway_immune, 'NES': enrich_immune, 'size':size_immune})
go_pathway_stat=pd.DataFrame({'pathway': pathway_go, 'NES': enrich_go, 'size':size_go})

In [105]:
one_hot_h=get_one_hot(heat_list_h, heat_dict_h, pathway_h)
one_hot_immune=get_one_hot(heat_list_immune, heat_dict_immune, pathway_immune)
one_hot_go=get_one_hot(heat_list_go, heat_dict_go, pathway_go)

In [106]:
h_heat=pd.DataFrame(one_hot_h)
immune_heat=pd.DataFrame(one_hot_immune)
go_heat=pd.DataFrame(one_hot_go)

In [112]:
immune_heat=immune_heat.set_index(pathway_immune)
h_heat=pd.DataFrame(pathway_h)
go_heat=pd.DataFrame(pathway_go)

In [110]:
H_gene_occ.to_csv("Hallmark_occ.csv")
immune_gene_occ.to_csv("immune_occ.csv")
go_gene_occ.to_csv("go_occ.csv")

In [114]:
H_pathway_stat.to_csv("Hallmark_stat.csv")
immune_pathway_stat.to_csv("immune_stat.csv")
go_pathway_stat.to_csv("go_stat.csv")

In [115]:
h_heat.to_csv("Hallmark_heat.csv")
immune_heat.to_csv("immune_heat.csv")
go_heat.to_csv("go_heat.csv")